In [2]:
%pip install tiktoken

  Using cached tiktoken-0.6.0-cp311-cp311-win_amd64.whl.metadata (6.8 kB)
  Using cached regex-2023.12.25-cp311-cp311-win_amd64.whl.metadata (41 kB)
Using cached tiktoken-0.6.0-cp311-cp311-win_amd64.whl (798 kB)
Using cached regex-2023.12.25-cp311-cp311-win_amd64.whl (269 kB)
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import tiktoken
from openai import OpenAI

# client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY", "<your OpenAI API key>"))
client = OpenAI()

In [4]:
titles = [
    "1. 생성형 AI의 열풍과 시장 확대 가. ChatGPT가 촉발한 생성형 AI 경쟁 나. 생성형 AI 시장 규모 다. 국내외 기업 및 기관의 생성형 AI 현황",
    "2. 생성형 AI의 효용성 가. 코드 생성, 문서화 및 품질 보증 나. 제품 개발 및 관리 다. 블로그 및 소셜 미디어 콘텐츠 작성 라. 인바운드 및 아웃바운드 마케팅 및 영업 마. 프로젝트 관리 및 운영 바. 그래픽 디자인 및 비디오 마케팅 사. 엔터테인먼트 미디어 생성 아. 성과 관리 및 코칭 자. 비즈니스 성과 보고 및 데이터 분석",
    "3. 생성형 AI의 한계 가. 기능적 측면 나. 윤리적 측면 다. 비용적 측면 라. 환경적 측면 마. 저작권 측면 바. 개인정보 측면",
    "4. 생성형 AI 활용 방안 가. 비즈니스에 생성형 AI 활용 현황 나. 생성형 AI 활용 시 주요 고려사항"
]

In [5]:
def parse_for_chat_history(response: client = False, user_prompt=None):
    if user_prompt:
        return {'role': 'user', 'content': user_prompt}
    return {'role': 'assistant', 'content': response.choices[0].message.content}

In [6]:
def num_tokens_from_messages(messages, model="gpt-3.5-turbo-0613"):
    """Return the number of tokens used by a list of messages."""
    try:
        encoding = tiktoken.encoding_for_model(model)
    except KeyError:
        print("Warning: model not found. Using cl100k_base encoding.")
        encoding = tiktoken.get_encoding("cl100k_base")
    if model in {
        "gpt-3.5-turbo-0613",
        "gpt-3.5-turbo-16k-0613",
        "gpt-4-0314",
        "gpt-4-32k-0314",
        "gpt-4-0613",
        "gpt-4-32k-0613",
        }:
        tokens_per_message = 3
        tokens_per_name = 1
    elif model == "gpt-3.5-turbo-0301":
        tokens_per_message = 4  # every message follows <|start|>{role/name}\n{content}<|end|>\n
        tokens_per_name = -1  # if there's a name, the role is omitted
    elif "gpt-3.5-turbo" in model:
        print("Warning: gpt-3.5-turbo may update over time. Returning num tokens assuming gpt-3.5-turbo-0613.")
        return num_tokens_from_messages(messages, model="gpt-3.5-turbo-0613")
    elif "gpt-4" in model:
        print("Warning: gpt-4 may update over time. Returning num tokens assuming gpt-4-0613.")
        return num_tokens_from_messages(messages, model="gpt-4-0613")
    else:
        raise NotImplementedError(
            f"""num_tokens_from_messages() is not implemented for model {model}. See https://github.com/openai/openai-python/blob/main/chatml.md for information on how messages are converted to tokens."""
        )
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3  # every reply is primed with <|start|>assistant<|message|>
    return num_tokens

In [7]:
messages = [
  {"role": "system", "content": "You are a helpful, pattern-following assistant that translates corporate jargon into plain English."},
  {"role": "system", "name":"example_user", "content": "New synergies will help drive top-line growth."},
  {"role": "system", "name": "example_assistant", "content": "Things working well together will increase revenue."},
  {"role": "system", "name":"example_user", "content": "Let's circle back when we have more bandwidth to touch base on opportunities for increased leverage."},
  {"role": "system", "name": "example_assistant", "content": "Let's talk later when we're less busy about how to do better."},
  {"role": "user", "content": "This late pivot means we don't have time to boil the ocean for the client deliverable."},
]

model = "gpt-3.5-turbo-0301"

print(f"{num_tokens_from_messages(messages, model)} prompt tokens counted.")
# Should show ~126 total_tokens

127 prompt tokens counted.


In [8]:
system_prompt = "너는 신문기사 작성 어시스턴트이야. 생성형 AI 활용 방안에 대한 기사를 작성하고 있어. 아래 [제목 목록]이 있어. 각 제목을 글의 시작점으로 사용해서 각 제목에 대해 짧은 단락을 작성해줘. .\n\n"
system_prompt += "[제목 목록] :\n"

for title in titles:
    system_prompt += f"{title}\n"

# chat history object
chat_history =  []

# Add in the system message:
chat_history.append({'role': 'system', 'content': system_prompt})

In [9]:
print(system_prompt)

너는 신문기사 작성 어시스턴트이야. 생성형 AI 활용 방안에 대한 기사를 작성하고 있어. 아래 [제목 목록]이 있어. 각 제목을 글의 시작점으로 사용해서 각 제목에 대해 짧은 단락을 작성해줘. .

[제목 목록] :
1. 생성형 AI의 열풍과 시장 확대 가. ChatGPT가 촉발한 생성형 AI 경쟁 나. 생성형 AI 시장 규모 다. 국내외 기업 및 기관의 생성형 AI 현황
2. 생성형 AI의 효용성 가. 코드 생성, 문서화 및 품질 보증 나. 제품 개발 및 관리 다. 블로그 및 소셜 미디어 콘텐츠 작성 라. 인바운드 및 아웃바운드 마케팅 및 영업 마. 프로젝트 관리 및 운영 바. 그래픽 디자인 및 비디오 마케팅 사. 엔터테인먼트 미디어 생성 아. 성과 관리 및 코칭 자. 비즈니스 성과 보고 및 데이터 분석
3. 생성형 AI의 한계 가. 기능적 측면 나. 윤리적 측면 다. 비용적 측면 라. 환경적 측면 마. 저작권 측면 바. 개인정보 측면
4. 생성형 AI 활용 방안 가. 비즈니스에 생성형 AI 활용 현황 나. 생성형 AI 활용 시 주요 고려사항



### 토큰 크기에 따라 메시지 삭제하기
- 토큰 수가 한도를 초과하면 마지막 메시지를 제거
- 채팅 기록이 늘어남에 따라 토큰 수가 줄어들도록 한다.

In [10]:
MAX_TOKEN_SIZE = 1000

In [11]:
# 각 제목에 대해 청크를 생성
for title in titles:
    # chat_history에 사용자 프롬프트 추가
    chat_history.append(
        {"role": "user", "content": f"{title}에 대한 짧은 단락을 작성해."}
    )
    print("-"*80)
    print(chat_history)
    print("-"*80)

    response = client.chat.completions.create(
        model="gpt-3.5-turbo", messages=chat_history
    )

    # ChatGPT response를  chat_history에 추가
    chat_history.append(parse_for_chat_history(response))

    # chat_history 토큰갯수가 MAX_TOKEN_SIZE보다 큰 경우, system 메시지가 아닌 가장 오래된 메시지를 제거
    while num_tokens_from_messages(chat_history, model) > MAX_TOKEN_SIZE:
        # system 메시지가 아닌 메시지 인덱스 찾기 
        non_system_msg_index = next(
            (i for i, msg in enumerate(chat_history) if msg["role"] != "system"), None
        )

        # 시스템 메시지가 아닌 메시지가 있는 경우 제거
        if non_system_msg_index is not None:
            chat_history.pop(non_system_msg_index)

    # response 출력
    print(response)

--------------------------------------------------------------------------------
[{'role': 'system', 'content': '너는 신문기사 작성 어시스턴트이야. 생성형 AI 활용 방안에 대한 기사를 작성하고 있어. 아래 [제목 목록]이 있어. 각 제목을 글의 시작점으로 사용해서 각 제목에 대해 짧은 단락을 작성해줘. .\n\n[제목 목록] :\n1. 생성형 AI의 열풍과 시장 확대 가. ChatGPT가 촉발한 생성형 AI 경쟁 나. 생성형 AI 시장 규모 다. 국내외 기업 및 기관의 생성형 AI 현황\n2. 생성형 AI의 효용성 가. 코드 생성, 문서화 및 품질 보증 나. 제품 개발 및 관리 다. 블로그 및 소셜 미디어 콘텐츠 작성 라. 인바운드 및 아웃바운드 마케팅 및 영업 마. 프로젝트 관리 및 운영 바. 그래픽 디자인 및 비디오 마케팅 사. 엔터테인먼트 미디어 생성 아. 성과 관리 및 코칭 자. 비즈니스 성과 보고 및 데이터 분석\n3. 생성형 AI의 한계 가. 기능적 측면 나. 윤리적 측면 다. 비용적 측면 라. 환경적 측면 마. 저작권 측면 바. 개인정보 측면\n4. 생성형 AI 활용 방안 가. 비즈니스에 생성형 AI 활용 현황 나. 생성형 AI 활용 시 주요 고려사항\n'}, {'role': 'user', 'content': '1. 생성형 AI의 열풍과 시장 확대 가. ChatGPT가 촉발한 생성형 AI 경쟁 나. 생성형 AI 시장 규모 다. 국내외 기업 및 기관의 생성형 AI 현황에 대한 짧은 단락을 작성해.'}]
--------------------------------------------------------------------------------
ChatCompletion(id='chatcmpl-8twmmDC6JEhEeKb6tHNJikN0hL8nw', choices=[Choice(finish_reason='stop', index=0, logprobs=None, 